# Customer
Grouping customers in various ways to understand their behavior and preferences.

## Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from common_functions import get_engine, read_data_return_df

## Connecting to the Database

In [ ]:
engine = get_engine()

## Reading in the Data

In [ ]:
query = "SELECT * FROM Customers"
customers = read_data_return_df(query, engine)

customers.head()

In [ ]:
query = "SELECT * FROM Orders"
orders = read_data_return_df(query, engine)

orders.head()

## Data Cleaning